In [1]:
import pandas as pd
import xarray as xr
import numpy as np

In [2]:
# Read Plant City station data

plant_city = pd.read_csv("s3://noaa-ghcn-pds/csv/by_station/USC00087205.csv",storage_options={"anon": True},dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},parse_dates=['DATE']).set_index('DATE')

plant_city['ELEMENT'].unique()

/tmp/ipykernel_16453/3194961485.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  plant_city = pd.read_csv("s3://noaa-ghcn-pds/csv/by_station/USC00087205.csv",storage_options={"anon": True},dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},parse_dates=['DATE']).set_index('DATE')


array(['TMAX', 'TMIN', 'PRCP', 'TOBS', 'SNOW', 'SNWD', 'WT16', 'WT08',
       'WT11', 'DAPR', 'MDPR', 'WT14', 'WT03', 'WT01', 'WT06', 'WT04'],
      dtype=object)

In [3]:
# Preview Plant City dataframe
plant_city

,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
DATE,,,,,,,
1892-09-01,USC00087205,TMAX,322,NaN,NaN,6,NaN
1892-09-02,USC00087205,TMAX,317,NaN,NaN,6,NaN
1892-09-03,USC00087205,TMAX,317,NaN,NaN,6,NaN
1892-09-04,USC00087205,TMAX,322,NaN,NaN,6,NaN
1892-09-05,USC00087205,TMAX,333,NaN,NaN,6,NaN
...,...,...,...,...,...,...,...
2025-12-10,USC00087205,PRCP,0,NaN,NaN,H,1600.0
2025-12-11,USC00087205,PRCP,0,NaN,NaN,H,1600.0
2025-12-12,USC00087205,PRCP,0,NaN,NaN,H,1600.0


Strawberries are planted around October 1 and ready for harvest by the end of January. What is the mean risk of frost and freeze, defined as the mean number of days per month over the period 1991-2020 that the temperature has been observed to be less than or equal to 32 and 28 degrees Fahrenheit, respectively, that might damage the plants for each month during the October - January period?

In [4]:
# Convert temperature values to °F for later use


# Mask rows where ELEMENT = TMAX or TMIN
mask = plant_city['ELEMENT'].isin(['TMAX', 'TMIN'])

# Convert °C to °F
plant_city.loc[mask, 'DATA_VALUE'] = (
    plant_city.loc[mask, 'DATA_VALUE'] / 10 * 9/5 + 32
)

/tmp/ipykernel_16453/3926282749.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[89.96 89.06 89.06 ... 44.06 50.   60.08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  plant_city.loc[mask, 'DATA_VALUE'] = (


In [5]:
# Filter dataframe for years and months requires

min_temp = plant_city[
    (plant_city['ELEMENT'] == 'TMIN') &
    ((plant_city.index.year >= 1991) & (plant_city.index.year <= 2020)) &
    (plant_city.index.month.isin([10, 11, 12, 1]))
].copy()

In [6]:
# Set indices

min_temp['year'] = min_temp.index.year
min_temp['month'] = min_temp.index.month


In [7]:
# Create dataframe that counts days with frost and freeze temperatures

counts = (
    min_temp
    .assign(
        frost = min_temp['DATA_VALUE'] <= 32,
        freeze = min_temp['DATA_VALUE'] <= 28
    )
    .groupby(['year', 'month'])
    .agg(
        days_frost=('frost', 'sum'),
        days_freeze=('freeze', 'sum')
    )
)

# Calculate mean frost and freeze days

monthly_means = counts.groupby('month').mean()


In [8]:
monthly_means

,days_frost,days_freeze
month,,
1,1.866667,0.500000
10,0.000000,0.000000
11,0.033333,0.000000
12,0.600000,0.166667


To begin to explore the seasonal to sub-seasonal prediction of freeze events at this site, using code you adapt from Module 4, we're going to try to relate these cold events to the El Nino Southern Oscillation (ENSO).  You have a hypothesis that ENSO is related to seasonal prediction of freeze events, but you don't know which region to choose for calculating your anomalies.  The problem is that there are many ENSO indicies that represent forcing across the eastern and central Pacific: which SST forcing region is most

In [9]:
# Establish ENSO regions

enso_regions = {
    'nino1_2': {'lat': slice(0, -10), 'lon': slice(270, 280)},
    'nino3':  {"lat": slice(5, -5),  'lon': slice(210, 270)},
    'nino3_4': {"lat": slice(5, -5),  'lon': slice(190, 240)},
    'nino4':  {'lat': slice(5, -5),  'lon': slice(160, 210)},
}


In [10]:
# Read in netcdf file with SST Anomilies

anoms = xr.open_dataset('1979_2024_SST_TCWV.nc')
print(anoms)

<xarray.Dataset> Size: 2GB
Dimensions:      (time: 552, latitude: 521, longitude: 721)
Coordinates:
  * time         (time) datetime64[ns] 4kB 1979-01-31 1979-02-28 ... 2024-12-31
  * latitude     (latitude) float32 2kB 65.0 64.75 64.5 ... -64.5 -64.75 -65.0
  * longitude    (longitude) float32 3kB 120.0 120.2 120.5 ... 299.5 299.8 300.0
    month        (time) int64 4kB ...
Data variables:
    sst_anomaly  (time, latitude, longitude) float32 829MB ...
    tcwv         (time, latitude, longitude) float32 829MB ...


In [12]:
# Function that extracts ENSO time series of each region

enso_ts = {}

for name, region in enso_regions.items():
    enso_ts[name] = (
        anoms['sst_anomaly']
        .sel(latitude=region['lat'], longitude=region['lon'])
        .mean(dim=['latitude', 'longitude'])
        .to_series()
    )


In [15]:
enso_ts

{'nino1_2': time
 1979-01-31   -0.109278
 1979-02-28   -1.105384
 1979-03-31   -0.916840
 1979-04-30    0.332670
 1979-05-31   -0.037388
                 ...   
 2024-08-31   -0.222497
 2024-09-30   -0.536838
 2024-10-31   -0.394105
 2024-11-30    0.109832
 2024-12-31   -0.018401
 Name: sst_anomaly, Length: 552, dtype: float32,
 'nino3': time
 1979-01-31   -0.277619
 1979-02-28   -0.449654
 1979-03-31   -0.389055
 1979-04-30   -0.101818
 1979-05-31   -0.035659
                 ...   
 2024-08-31   -0.259784
 2024-09-30   -0.270204
 2024-10-31   -0.295238
 2024-11-30   -0.185195
 2024-12-31   -0.433527
 Name: sst_anomaly, Length: 552, dtype: float32,
 'nino3_4': time
 1979-01-31    0.012250
 1979-02-28    0.150540
 1979-03-31    0.160503
 1979-04-30    0.284099
 1979-05-31    0.244655
                 ...   
 2024-08-31    0.036564
 2024-09-30   -0.325694
 2024-10-31   -0.459476
 2024-11-30   -0.212379
 2024-12-31   -0.687500
 Name: sst_anomaly, Length: 552, dtype: float32,
 'nino4': ti

In [ ]:
# Function that finds mean values by grouped months in each ENSO region. Annual winter indices.

enso_wint = {}

for k, v in enso_ts.items():
    v_wint = v[v.index.month.isin([10, 11, 12, 1])]
    enso_wint[k] = v_wint.groupby(v_wint.index.year).mean()



In [ ]:
# Create dataframe that groups frost and freeze by year


days_frost_freeze_yearly = (
    counts
    .groupby(level='year')
    .sum()
)

days_frost_freeze_yearly.head()


,days_frost,days_freeze
year,,
1991,0,0
1992,3,0
1993,1,0
1994,0,0
1995,4,3


In [20]:
# Find years in common 

common_years = days_frost_freeze_yearly.index.intersection(
    enso_wint["nino3_4"].index
)

freeze = days_frost_freeze_yearly.loc[common_years]


In [22]:
# Function that finds correclation betweens ENSO winter indices and freeze and frost years in florida.

for name, enso in enso_wint.items():
    enso = enso.loc[common_years]
    r_frost  = enso.corr(freeze['days_frost'])
    r_freeze = enso.corr(freeze['days_freeze'])
    print(f"{name}: frost={r_frost:.2f}, freeze={r_freeze:.2f}")


nino1_2: frost=-0.33, freeze=-0.19
nino3: frost=-0.29, freeze=-0.26
nino3_4: frost=-0.25, freeze=-0.27
nino4: frost=-0.27, freeze=-0.34


The ENSO region with the highest correlation is Nino 4.